In [112]:


import redis
import pickle
import uuid
import time
import threading
import numpy as np


class KNNQuerySender:
    def __init__(self):
        self.redis_client = redis.StrictRedis(host='localhost', port=6379, db=0, password='secret_key')
        self.query_channel = 'knn_query_channel'
        self.response_channel = 'knn_query_response_channel'
        self.responses = {}
        self._listen_thread = threading.Thread(target=self.listen_for_responses)
        self._listen_thread.start()

    def send_query(self, sample, distance, voting_method):
        request_id = str(uuid.uuid4())
        query_data = {
            'sample': sample,
            'distance': distance,
            'voting_method': voting_method,
            'request_id': request_id
        }
        self.redis_client.publish(self.query_channel, pickle.dumps(query_data))
        return request_id

    def listen_for_responses(self):
        pubsub = self.redis_client.pubsub()
        pubsub.subscribe(self.response_channel)
        for message in pubsub.listen():
            if message['type'] == 'message':
                response_data = pickle.loads(message['data'])
                request_id = response_data['request_id']
                result = response_data['result']
                self.responses[request_id] = result

    def get_response(self, request_id):
        while request_id not in self.responses:
            time.sleep(0.1)  # Sleep briefly to avoid busy-waiting
        return self.responses.pop(request_id)

def test(distance ,voting_method ):
    
    predictions = []
    for sample in X_test:
        request_id = query_sender.send_query(sample, distance, voting_method)
        final_prediction = query_sender.get_response(request_id)
        predictions.append(final_prediction)

    accuracy = sum(np.array(predictions) == y_test) / len(y_test)
    print(f'Accuracy for distance {distance} and voting method {voting_method}: {accuracy*100}')
    
    
query_sender = KNNQuerySender()

X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')


In [49]:
X_test.shape

(360, 64)

In [50]:
test(distance = 'euclidean',voting_method = 'weighted_voting')

Accuracy: 98.33333333333333


In [51]:
test(distance = 'cosine',voting_method = 'weighted_voting')

Accuracy: 98.88888888888889


In [60]:
test(distance = 'manhattan',voting_method = 'weighted_voting')

Accuracy: 97.77777777777777


In [53]:
test(distance = 'euclidean',voting_method = 'majority_voting')

Accuracy: 98.33333333333333


In [54]:
test(distance = 'cosine',voting_method = 'majority_voting')

Accuracy: 98.88888888888889


In [57]:
test(distance = 'euclidean',voting_method = 'distance_based_aggregation')

Accuracy: 98.33333333333333


In [58]:
test(distance = 'cosine',voting_method = 'distance_based_aggregation')

Accuracy: 98.88888888888889


In [59]:
test(distance = 'manhattan',voting_method = 'distance_based_aggregation')

Accuracy: 97.77777777777777


In [80]:
# With tampering and no defense aganist tampering
test(distance = 'cosine',voting_method = 'distance_based_aggregation')

Accuracy: 97.5


In [81]:
test(distance = 'cosine',voting_method = 'majority_voting')

Accuracy: 98.05555555555556


In [ ]:
test(distance = 'cosine',voting_method = 'majority_voting')

In [92]:
test(distance = 'cosine',voting_method = 'min_distance')

Accuracy for distance cosine and voting method min_distance: 98.33333333333333


In [100]:
distances = ['cosine' ,'manhattan','euclidean']
voting_methods = ['distance_based_aggregation','majority_voting','weighted_voting','min_distance']

In [85]:
# With Tampering
for distance in distances:
    for voting_method in voting_methods:
        test(distance = distance,voting_method = voting_method)

Accuracy for distance cosine and voting method distance_based_aggregation: 97.5
Accuracy for distance cosine and voting method majority_voting: 98.05555555555556
Accuracy for distance cosine and voting method weighted_voting: 98.33333333333333
Accuracy for distance cosine and voting method min_distance: 91.38888888888889
Accuracy for distance manhattan and voting method distance_based_aggregation: 97.22222222222221
Accuracy for distance manhattan and voting method majority_voting: 97.77777777777777
Accuracy for distance manhattan and voting method weighted_voting: 97.5
Accuracy for distance manhattan and voting method min_distance: 90.0
Accuracy for distance euclidean and voting method distance_based_aggregation: 97.77777777777777
Accuracy for distance euclidean and voting method majority_voting: 98.33333333333333
Accuracy for distance euclidean and voting method weighted_voting: 98.05555555555556
Accuracy for distance euclidean and voting method min_distance: 91.11111111111111


In [90]:
# With Tampering and mitigation applied
for distance in distances:
    for voting_method in voting_methods:
        test(distance = distance,voting_method = voting_method)

Accuracy for distance cosine and voting method distance_based_aggregation: 96.94444444444444
Accuracy for distance cosine and voting method majority_voting: 97.5
Accuracy for distance cosine and voting method weighted_voting: 98.05555555555556
Accuracy for distance cosine and voting method min_distance: 97.22222222222221
Accuracy for distance manhattan and voting method distance_based_aggregation: 96.11111111111111
Accuracy for distance manhattan and voting method majority_voting: 96.38888888888889
Accuracy for distance manhattan and voting method weighted_voting: 96.66666666666667
Accuracy for distance manhattan and voting method min_distance: 96.66666666666667
Accuracy for distance euclidean and voting method distance_based_aggregation: 97.22222222222221
Accuracy for distance euclidean and voting method majority_voting: 97.5
Accuracy for distance euclidean and voting method weighted_voting: 97.5
Accuracy for distance euclidean and voting method min_distance: 96.94444444444444


In [13]:
# With tampering and rate limiting applied, it was executed immediately
sample = [ 0.,  0.,  0.,  7., 12.,  0.,  0.,  0.,  0.,  0.,  4., 16.,  8.,
        0.,  0.,  0.,  0.,  0., 12., 11.,  0.,  0.,  0.,  0.,  0.,  0.,
       15., 10.,  8.,  6.,  1.,  0.,  0.,  0., 15., 16.,  8., 10.,  8.,
        0.,  0.,  0., 14.,  7.,  0.,  0., 12.,  0.,  0.,  0.,  8., 11.,
        0.,  5., 16.,  2.,  0.,  0.,  0.,  9., 14., 14.,  5.,  0.]
distance = 'cosine'
voting_method = 'majority_voting'
request_id = query_sender.send_query(sample, distance, voting_method)
print(request_id)
result = query_sender.get_response(request_id)
print(f"Result for request {request_id}: {result}")

62b8a3fa-c81b-4e0e-aef8-5259b0561fe7
Result for request 62b8a3fa-c81b-4e0e-aef8-5259b0561fe7: 6


In [164]:
# With no rate limiting applied , it waas executed in 18 minutes
sample = [ 0.,  0.,  0.,  7., 12.,  0.,  0.,  0.,  0.,  0.,  4., 16.,  8.,
        0.,  0.,  0.,  0.,  0., 12., 11.,  0.,  0.,  0.,  0.,  0.,  0.,
       15., 10.,  8.,  6.,  1.,  0.,  0.,  0., 15., 16.,  8., 10.,  8.,
        0.,  0.,  0., 14.,  7.,  0.,  0., 12.,  0.,  0.,  0.,  8., 11.,
        0.,  5., 16.,  2.,  0.,  0.,  0.,  9., 14., 14.,  5.,  0.]
distance = 'cosine'
voting_method = 'majority_voting'
request_id = query_sender.send_query(sample, distance, voting_method)
print(request_id)
result = query_sender.get_response(request_id)
print(f"Result for request {request_id}: {result}")

d988f3c7-1748-4fc0-a50a-1d0f37b84ce2
Result for request d988f3c7-1748-4fc0-a50a-1d0f37b84ce2: 6


In [113]:
distances = ['cosine' ,'manhattan','euclidean']
voting_methods = ['distance_based_aggregation','majority_voting','weighted_voting','min_distance']
# Spoofing is up
# With Tampering
# expected = 6
sample = [ 0.,  0.,  0.,  7., 12.,  0.,  0.,  0.,  0.,  0.,  4., 16.,  8.,
        0.,  0.,  0.,  0.,  0., 12., 11.,  0.,  0.,  0.,  0.,  0.,  0.,
       15., 10.,  8.,  6.,  1.,  0.,  0.,  0., 15., 16.,  8., 10.,  8.,
        0.,  0.,  0., 14.,  7.,  0.,  0., 12.,  0.,  0.,  0.,  8., 11.,
        0.,  5., 16.,  2.,  0.,  0.,  0.,  9., 14., 14.,  5.,  0.]
for distance in distances:
    for voting_method in voting_methods:
        request_id = query_sender.send_query(sample, distance, voting_method)
       
        result = query_sender.get_response(request_id)
        print(f'Result for request with distance {distance} and voting method {voting_method} is : {result}')



Result for request with distance cosine and voting method distance_based_aggregation is : 8
Result for request with distance cosine and voting method majority_voting is : 6
Result for request with distance cosine and voting method weighted_voting is : 6
Result for request with distance cosine and voting method min_distance is : 8
Result for request with distance manhattan and voting method distance_based_aggregation is : 8
Result for request with distance manhattan and voting method majority_voting is : 6
Result for request with distance manhattan and voting method weighted_voting is : 6
Result for request with distance manhattan and voting method min_distance is : 8
Result for request with distance euclidean and voting method distance_based_aggregation is : 8
Result for request with distance euclidean and voting method majority_voting is : 6
Result for request with distance euclidean and voting method weighted_voting is : 6
Result for request with distance euclidean and voting method 

Here we simulated a spoofing node that upon receving req, send a prediction for class 8 and of distances =0 thats why distnace_based_aggregation and min_distance were affected

In [115]:
distances = ['cosine' ,'manhattan','euclidean']
voting_methods = ['distance_based_aggregation','majority_voting','weighted_voting','min_distance']
# test here 
# expected = 6
sample = [ 0.,  0.,  0.,  7., 12.,  0.,  0.,  0.,  0.,  0.,  4., 16.,  8.,
        0.,  0.,  0.,  0.,  0., 12., 11.,  0.,  0.,  0.,  0.,  0.,  0.,
       15., 10.,  8.,  6.,  1.,  0.,  0.,  0., 15., 16.,  8., 10.,  8.,
        0.,  0.,  0., 14.,  7.,  0.,  0., 12.,  0.,  0.,  0.,  8., 11.,
        0.,  5., 16.,  2.,  0.,  0.,  0.,  9., 14., 14.,  5.,  0.]
for distance in distances:
    for voting_method in voting_methods:
        request_id = query_sender.send_query(sample, distance, voting_method)
       
        result = query_sender.get_response(request_id)
        print(f'Result for request with distance {distance} and voting method {voting_method} is : {result}')



Result for request with distance cosine and voting method distance_based_aggregation is : 6
Result for request with distance cosine and voting method majority_voting is : 6
Result for request with distance cosine and voting method weighted_voting is : 6
Result for request with distance cosine and voting method min_distance is : 6
Result for request with distance manhattan and voting method distance_based_aggregation is : 6
Result for request with distance manhattan and voting method majority_voting is : 6
Result for request with distance manhattan and voting method weighted_voting is : 6
Result for request with distance manhattan and voting method min_distance is : 6
Result for request with distance euclidean and voting method distance_based_aggregation is : 6
Result for request with distance euclidean and voting method majority_voting is : 6
Result for request with distance euclidean and voting method weighted_voting is : 6
Result for request with distance euclidean and voting method 

In [114]:
X_test[6]

array([ 0.,  0.,  8., 12., 16., 16.,  9.,  0.,  0.,  4., 16., 16., 13.,
        9.,  2.,  0.,  0., 11., 14.,  4.,  0.,  0.,  0.,  0.,  0.,  7.,
       15., 10.,  1.,  0.,  0.,  0.,  0.,  0., 12., 16., 13.,  1.,  0.,
        0.,  0.,  0.,  0.,  4., 16.,  4.,  0.,  0.,  0.,  0.,  5., 10.,
       16.,  3.,  0.,  0.,  0.,  0.,  9., 16., 10.,  0.,  0.,  0.])